In [ ]:
filePath = "B:/enzymeClass/transaminaseProtein.fasta"

def fastLength(filePath):
    fastaLen = 0
    with open(filePath, 'r') as fasta:
        for line in fasta.readlines():
            if '>' in line:
                fastaLen += 1
    return fastaLen

def readMultipleFast(filePath):
    header = []
    seq = ''
    with open(filePath, 'r') as fast:
        for line in fast.readlines():
            if '>' in line:
                header.append(line)
            else: seq += line
    multifast = seq.split('\n\n')
    for s in range(len(multifast)):
        multifast[s] = multifast[s].replace('\n', '')
    return multifast[:-1], header

print('length of fasta file or numbers of reads: %d'%fastLength(filePath))
fasta, header = readMultipleFast(filePath)

In [3]:
################ SAMPLE SEQUENCE ################
seq2 = 'MLARALLLCAVLALSHTANPCCSHPCQNRGVCMSVGFDQYKCDCTRTGFYGENCSTPEFLTRIKLFLKPTPNTVHYILTHFKGFWNVVNNIPFLRNAIMSYVLTSRSHLIDSPPTYNADYGYKSWEAFSNLSYYTRALPPVPDDCPTPLGVKGKKQLPDSNEIVEKLLLRRKFIPDPQGSNMMFAFFAQHFTHQFFKTDHKRGPAFTNGLGHGVDLNHIYGETLARQRKLRLFKDGKMKYQIIDGEMYPPTVKDTQAEMIYPPQVPEHLRFAVGQEVFGLVPGLMMYATIWLREHNRVCDVLKQEHPEWGDEQLFQTSRLILIGETIKIVIEDYVQHLSGYHFKLKFDPELLFNKQFQYQNRIAAEFNTLYHWHPLLPDTFQIHDQKYNYQQFIYNNSILLEHGITQFVESFTRQIAGRVAGGRNVPPAVQKVSQASIDQSRQMKYQSFNEYRKRFMLKPYESFEELTGEKEMSAELEALYGDIDAVELYPALLVEKPRPDAIFGETMVEVGAPFSLKGLMGNVICSPAYWKPSTFGGEVGFQIINTASIQSLICNNVKGCPFTSFSVPDPELIKTVTINASSSRSGLDDINPTVLLKERSTEL'

In [ ]:
ulis = []
stri = ''
for i in seq2:
    if i not in ulis:
        ulis.append(i)
for i in ulis:
    stri+=i
stri

In [6]:
import numpy as np
import matplotlib.pyplot as plt

seq1 = 'ATGCTCGTGCTATACGGA'

def One_Hot_encoding(string, ALIGN_CONST='ATGC'):
    encoded = []
    for i in ALIGN_CONST:
        row = []
        for j in string:
            if i == j:
                row.append(1.0)
            else: row.append(0.0)
        encoded.append(row)
    return encoded

def bandImage_Genretor(imageArray, fileName, dpi=100, _max=200, cmap='gray'):
    if _max > dpi:
        plt.figure(figsize=(int(_max/dpi), int(_max/dpi)), dpi=dpi)
        plt.imshow(imageArray, interpolation='nearest', cmap=cmap, aspect='auto')
        plt.axis('off')
        plt.savefig(fileName, pad_inches=0, bbox_inches='tight')
        return "saved -> "+fileName
    else: return '_max value not greater than dpi..'

In [ ]:
# Image Dimensions: 155x154
# Format: PNG 
for seq in range(len(fasta)):
    imgarr = np.array(One_Hot_encoding(fasta[seq], ALIGN_CONST='MLARCVSHTNPQGFDYKEIW'))
    bandImage_Genretor(imgarr, 'B:/enzymeClass/DataImage/transaminase/amin%d.png'%seq)

In [6]:
import tensorflow as tf
from keras import layers, models

train_dir = "B:/enzymeClass/DataImage"
train = tf.keras.preprocessing.image_dataset_from_directory(train_dir, image_size=(155, 154), color_mode="grayscale", shuffle=True)
train

Found 6960 files belonging to 5 classes.


<_BatchDataset element_spec=(TensorSpec(shape=(None, 155, 154, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [7]:
model = models.Sequential(
    [layers.Conv2D(32, (3, 3), activation='relu', input_shape=(155, 154, 1)),
     layers.MaxPooling2D(),
     layers.Conv2D(64, (3, 3), activation='relu'),
     layers.MaxPooling2D(),
     layers.Conv2D(64, (3, 3), activation='relu'),
     layers.MaxPooling2D(),
     layers.Conv2D(128, (3, 3), activation='relu'),
     layers.MaxPooling2D(),
     layers.Conv2D(128, (3, 3), activation='relu'),
     layers.MaxPooling2D(),
     layers.Flatten(),
     layers.Dropout(0.5),
     layers.Dense(512, activation='relu'),
     layers.Dense(5, activation='softmax')
    ]
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 153, 152, 32)      320       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 76, 76, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 74, 74, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 35, 35, 64)        36928     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 17, 17, 64)      

In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
modelrun = model.fit(train, epochs=2)

Epoch 1/2
218/218 [==============================] - 132s 593ms/step - loss: 1.4079 - accuracy: 0.5629
Epoch 2/2
218/218 [==============================] - 127s 581ms/step - loss: 0.9789 - accuracy: 0.6174


In [11]:
model.save_weights('weight.h5')
model.layers[13].get_weights()

[array([[ 0.0049341 , -0.05735083, -0.09809205,  0.08885757, -0.09375548],
        [-0.00769438, -0.05581054,  0.05634931, -0.00436593, -0.00147459],
        [ 0.10049343,  0.10113937,  0.02817721, -0.07018766, -0.04238156],
        ...,
        [ 0.10272818, -0.08063234,  0.01502476,  0.09140489, -0.10593095],
        [ 0.07181194,  0.08580116,  0.07371862,  0.06823976,  0.0688099 ],
        [-0.06214504,  0.04888536,  0.04122535,  0.07611465,  0.05051546]],
       dtype=float32),
 array([ 0.01354859,  0.00398914, -0.03905588,  0.01513773, -0.0462256 ],
       dtype=float32)]

In [12]:
testdir = 'B:/enzymeClass/testimages'
test = tf.keras.preprocessing.image_dataset_from_directory(testdir, image_size=(155, 154), color_mode="grayscale", shuffle=True)
model.evaluate(test)

Found 1298 files belonging to 5 classes.
41/41 [==============================] - 7s 155ms/step - loss: 1.6105 - accuracy: 0.4168


[1.6104521751403809, 0.4167950749397278]

In [ ]:
##################### TARGET PROTEIN AND GENE SEQUENCE SCRAPING #####################

from bs4 import BeautifulSoup
import requests

uniProtID = 'P35354'
folderLocation = "A:/PROJECTS/BIOINFORMATICS_software/TargetsData"

# 'class': 'col-xl-10 col-md-9 col-sm-8', 'tag': 'pre'
url = f'https://go.drugbank.com/polypeptides/{uniProtID}'
soup = BeautifulSoup(requests.get(url).text, 'xml')
target = soup.find('dd', {'class': 'col-xl-10 col-md-9 col-sm-8'}).text
if '/' in target:
    trName = target.replace('/', '_')
else: trName = target
val = soup.find_all('pre')
with open(f"{folderLocation}/{trName}.fasta", "w") as file:
    for item in val:
        file.write(item.text)
        file.write('\n')

In [65]:
# SEGMENTATION CODE #
seq = 'GYKSWEAFSNLSYYTRALPPVPDDCPTPLGVKGKKQLPDSNEIVEKLLLRRKFIPDPQGS'

def segmentation(seq, seg_Length=5):
    segment = []
    for i in range(0, (len(seq)-seg_Length)+1):
        segment.append(seq[i:i+seg_Length])
    return segment

In [ ]:
# SEGMENTED GC REGION #
seq = 'GYKSWEAFSNLSYYTRALPPVPDDCPTPLGVKGKKQLPDSNEIVEKLLLRRKFIPDPQGS'
se = 5
threshold = 2.0

(pos, pos_l) = ('', [])
gc_percent = []
for seg in segmentation(seq, seg_Length=se):
    (_C, _G) = (seg.count('C'), seg.count('G'))
    gc_percent.append(_C+_G/se)
for i in range(len(gc_percent)):
    if threshold > min(gc_percent):
        th = threshold
    else: th = 2.0
    if gc_percent[i] >= min(gc_percent)+th:
        pos += '{},'.format(i)
        pos_l.append(i)
    else: pos += '~'

pos, pos_l